# Procedural REPNET Analysis

In [ ]:
# Import needed modules and define epsilon
import collections
import random
import gym
import numpy as np
import statistics
import tensorflow as tf
import tqdm
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from typing import List, Tuple

# Define epsilon through numpy
eps = np.finfo(np.float32).eps.item()

Create the basic Actor-Critic model:



In [ ]:
class ActorCritic(tf.keras.Model):
    """Combined actor_critic network."""

    def __init__(
            self,
            num_actions: int,
            num_hidden_units: int):
        """Initialize."""
        super().__init__()

        self.common = layers.Dense(num_hidden_units, activation="relu")
        self.actor = layers.Dense(num_actions)
        self.critic = layers.Dense(1)

    def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
        x = self.common(inputs)
        return self.actor(x), self.critic(x)


class CartPoleEnv:
    def __init__(self):
        self.env = gym.make("CartPole-v1")

    def env_step(self, action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Returns state, reward and done flag given an action."""

        state, reward, done, _ = self.env.step(action)
        return (state.astype(np.float32),
                np.array(reward, np.int32),
                np.array(done, np.int32))

    def tf_env_step(self, action: tf.Tensor) -> List[tf.Tensor]:
        return tf.numpy_function(self.env_step, [action],
                                 [tf.float32, tf.int32, tf.int32])

    def run_episode(self,
            initial_state: tf.Tensor,
            model: tf.keras.Model,
            max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
        """Runs a single episode to collect training data."""

        action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

        initial_state_shape = initial_state.shape
        state = initial_state

        for t in tf.range(max_steps):
            # Convert state into a batched tensor (batch size = 1)
            state = tf.expand_dims(state, 0)

            # Run the model and to get action probabilities and critic value
            action_logits_t, value = model(state)

            # Sample next action from the action probability distribution
            action = tf.random.categorical(action_logits_t, 1)[0, 0]
            action_probs_t = tf.nn.softmax(action_logits_t)

            # Store critic values
            values = values.write(t, tf.squeeze(value))

            # Store log probability of the action chosen
            action_probs = action_probs.write(t, action_probs_t[0, action])

            # Apply action to the environment to get next state and reward
            state, reward, done = self.tf_env_step(action)
            state.set_shape(initial_state_shape)

            # Store reward
            rewards = rewards.write(t, reward)

            if tf.cast(done, tf.bool):
                break

        action_probs = action_probs.stack()
        values = values.stack()
        rewards = rewards.stack()

        return action_probs, values, rewards

    def get_expected_return(
            self,
            rewards: tf.Tensor,
            gamma: float,
            standardize: bool = True) -> tf.Tensor:
        """Compute expected returns per timestep."""

        n = tf.shape(rewards)[0]
        returns = tf.TensorArray(dtype=tf.float32, size=n)

        # Start from the end of `rewards` and accumulate reward sums
        # into the `returns` array
        rewards = tf.cast(rewards[::-1], dtype=tf.float32)
        discounted_sum = tf.constant(0.0)
        discounted_sum_shape = discounted_sum.shape
        for i in tf.range(n):
            reward = rewards[i]
            discounted_sum = reward + gamma * discounted_sum
            discounted_sum.set_shape(discounted_sum_shape)
            returns = returns.write(i, discounted_sum)
        returns = returns.stack()[::-1]

        if standardize:
            returns = ((returns - tf.math.reduce_mean(returns)) /
                       (tf.math.reduce_std(returns) + eps))

        return returns

    def compute_loss(
            self,
            action_probs: tf.Tensor,
            values: tf.Tensor,
            returns: tf.Tensor) -> tf.Tensor:
        """Computes the combined actor_critic loss."""
        huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

        advantage = returns - values

        action_log_probs = tf.math.log(action_probs)
        actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

        critic_loss = huber_loss(values, returns)

        return actor_loss + critic_loss

    @tf.function
    def train_step(
            self,
            initial_state: tf.Tensor,
            model: tf.keras.Model,
            optimizer: tf.keras.optimizers.Optimizer,
            gamma: float,
            max_steps_per_episode: int) -> tf.Tensor:
        """Runs a model training step."""

        with tf.GradientTape() as tape:
            # Run the model for one episode to collect training data
            action_probs, values, rewards = self.run_episode(
                initial_state, model, max_steps_per_episode)

            # Calculate expected returns
            returns = self.get_expected_return(rewards, gamma)

            # Convert training data to appropriate TF tensor shapes
            action_probs, values, returns = [
                tf.expand_dims(x, 1) for x in [action_probs, values, returns]]

            # Calculating loss values to update our network
            loss = self.compute_loss(action_probs, values, returns)

        # Compute the gradients from the loss
        grads = tape.gradient(loss, model.trainable_variables)

        # Apply the gradients to the model's parameters
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        episode_reward = tf.math.reduce_sum(rewards)

        return episode_reward

Create the REPNET architecture:

In [ ]:
class MinBranch:
    def __init__(self, max_kill_iters, threshold_func, performance=0, weights=None, generation=0):
        self.max_kill_iters = max_kill_iters
        self.reset_iters = 0

        self.threshold_func = threshold_func
        self.performance = performance
        self.weights = weights
        self.child_branches = []
        self.killed = False

        self.generation = generation

    def check_kill(self):
        if self.reset_iters > self.max_kill_iters:
            self.killed = True

    def get_branch_num_ends(self):
        if len(self.child_branches) == 0:
            return 1

        branch_end_sum = 0
        for child in self.child_branches:
            branch_end_sum += child.get_branch_num_ends()
        return branch_end_sum

    def get_branch_ends(self):
        if len(self.child_branches) == 0:
            return [self]

        branch_ends = []
        for child in self.child_branches:
            branch_ends.append(child.get_branch_ends())
        return np.array(branch_ends).flatten()

    def update(self, new_performance, new_weights):
        self.check_kill()
        if self.killed:
            # del self
            return True

        if new_performance - self.performance > self.threshold_func(self.performance):
            self.child_branches.append(MinBranch(self.max_kill_iters,
                                                 self.threshold_func,
                                                 performance=new_performance,
                                                 weights=new_weights,
                                                 generation=self.generation + 1
                                                 ))
            self.reset_iters = 0
        self.reset_iters += 1
        return False

    def __str__(self):
        ret_str = ""

        # Check and see if main branch, if so specify
        if self.generation == 1:
            ret_str += "trunk: "
        else:
            ret_str += "branch: "

        if not self.killed:
            ret_str += f"Generation {self.generation}," \
                       f"Performance: {self.performance}, Iterations since reset: {self.reset_iters}\n"
        for branch in self.child_branches:
            for i in range(self.generation):
                ret_str += "    "
            ret_str += "↳" + branch.__str__()

        return ret_str

class TreeRL:
    def __init__(self, max_kill_iters, threshold_func, model_baseline):
        self.main = MinBranch(max_kill_iters, threshold_func)
        self.model_baseline = model_baseline

        self.best_weights = None
        self.best_performance = 0

    def get_num_branch_ends(self):
        return self.main.get_branch_num_ends()

    def get_branch_ends(self):
        return self.main.get_branch_ends()

    def update_end(self, end, performance, weights):
        if performance > self.best_performance:
            self.best_weights = weights
            self.best_performance = performance
        return end.update(performance, weights)

Create the training functions:

In [ ]:
# Training functions

def train_repnet(threshold_func=lambda x: 100, kill_time=100, gamma=0.99, max_episodes=10000, learning_rate=0.01,
                 reward_threshold=195, seed=None):
    """
    Trains a basic REPNET
    :param seed: the randomizng seed to train on
    :param threshold_func: the input function to deterimine when to create a new branch
    :param kill_time: the maximum number of episodes from the last time of reproduction
    that a new branch can exist for until it dies
    :param gamma: the discount factor for future rewards
    :param max_episodes: the maximum amount of episodes that can be trained until the training session exits (hard stop)
    :param learning_rate: The learning rate of the Adam optimizer
    :param reward_threshold: The minimum running reward needed to consider the model trained
    :return: A list of the running reward every episode
    """
    cartpole = CartPoleEnv()

    if seed is None:
        seed = random.randrange(0, 100)
    # cartpole.env.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

    min_episodes_criterion = 140
    max_steps_per_episode = 1000

    # Cartpole-v0 is considered solved if average reward is >= 195 over 100
    # consecutive trials
    running_reward = 0

    # Keep last episodes reward
    episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

    num_actions = cartpole.env.action_space.n  # 2
    num_hidden_units = 128

    model = ActorCritic(num_actions, num_hidden_units)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    trunk = TreeRL(kill_time, threshold_func, model)

    for end in trunk.get_branch_ends():
        end.weights = model.get_weights()

    performance_list = []

    iters = 0
    for i in range(max_episodes):
        # print(trunk.get_branch_ends())
        for end in trunk.get_branch_ends():
            if end.killed:
                continue

            if end.weights is not None:
                model.set_weights(end.weights)
            iters += 1

            initial_state = tf.constant(cartpole.env.reset(), dtype=tf.float32)
            episode_reward = int(cartpole.train_step(
                initial_state, model, optimizer, gamma, max_steps_per_episode))

            episodes_reward.append(episode_reward)
            running_reward = statistics.mean(episodes_reward)

            performance_list.append(running_reward)

            end.weights = model.get_weights()

            if running_reward > reward_threshold:
                return performance_list

            trunk.update_end(end, episode_reward, model.get_weights())
    if performance_list[-1] < reward_threshold:
        return train_repnet()
    print(len(performance_list))
    return performance_list


def train_distributed_repnet(threshold_func=lambda x: 100, kill_time=100, gamma=0.99, max_episodes=10000,
                             learning_rate=0.01, reward_threshold=195, seed=None):
    """
    Trains REPNET based on the multiprocessing distribution principle
    :param seed: the randomizing seed to train on
    :param threshold_func: the input function to deterimine when to create a new branch
    :param kill_time: the maximum number of episodes from the last time of reproduction
    that a new branch can exist for until it dies
    :param gamma: the discount factor for future rewards
    :param max_episodes: the maximum amount of episodes that can be trained until the training session exits (hard stop)
    :param learning_rate: The learning rate of the Adam optimizer
    :param reward_threshold: The minimum running reward needed to consider the model trained
    :return: A list of the running reward every episode
    """
    cartpole = CartPoleEnv()

    if seed is None:
        seed = random.randrange(0, 100)
    # cartpole.env.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

    min_episodes_criterion = 140
    max_steps_per_episode = 1000

    # Cartpole-v0 is considered solved if average reward is >= 195 over 100
    # consecutive trials
    running_reward = 0

    # Keep last episodes reward
    episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

    num_actions = cartpole.env.action_space.n  # 2
    num_hidden_units = 128

    model = ActorCritic(num_actions, num_hidden_units)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    trunk = TreeRL(kill_time, threshold_func, model)

    for end in trunk.get_branch_ends():
        end.weights = model.get_weights()

    performance_list = []

    iters = 0
    for i in range(max_episodes):
        performance_list.append(running_reward)
        for end in trunk.get_branch_ends():
            if end.killed:
                continue

            if end.weights is not None:
                model.set_weights(end.weights)
            iters += 1

            initial_state = tf.constant(cartpole.env.reset(), dtype=tf.float32)
            episode_reward = int(cartpole.train_step(
                initial_state, model, optimizer, gamma, max_steps_per_episode))

            episodes_reward.append(episode_reward)
            running_reward = statistics.mean(episodes_reward)

            end.weights = model.get_weights()

            if running_reward > reward_threshold:
                return performance_list

            trunk.update_end(end, episode_reward, model.get_weights())
    if performance_list[-1] < reward_threshold:
        return train_distributed_repnet()
    print(len(performance_list))
    return performance_list


def train_rl(gamma=0.99, max_episodes=10000, learning_rate=0.01, reward_threshold=195, seed=None):
    """
    Trains a normal actor-critic network
    :param seed: The randomizing seed to train on
    :param gamma: the discount factor for future rewards
    :param max_episodes: the maximum amount of episodes that can be trained until the training session exits (hard stop)
    :param learning_rate: The learning rate of the Adam optimizer
    :param reward_threshold: The minimum running reward needed to consider the model trained
    :return: A list of the running reward every episode
    """
    cartpole = CartPoleEnv()

    if seed is None:
        seed = random.randrange(0, 100)
    # cartpole.env.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

    # Small epsilon value for stabilizing division operations
    eps = np.finfo(np.float32).eps.item()

    min_episodes_criterion = 100
    max_steps_per_episode = 1000

    # Cartpole-v0 is considered solved if average reward is >= 195 over 100
    # consecutive trials
    running_reward = 0

    # Keep last episodes reward
    episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

    num_actions = cartpole.env.action_space.n  # 2
    num_hidden_units = 128

    model = ActorCritic(num_actions, num_hidden_units)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    performance_list = []

    iters = 0

    with tqdm.trange(max_episodes) as t:
        for i in t:
            initial_state = tf.constant(cartpole.env.reset(), dtype=tf.float32)
            episode_reward = int(cartpole.train_step(
                initial_state, model, optimizer, gamma, max_steps_per_episode))

            episodes_reward.append(episode_reward)
            running_reward = statistics.mean(episodes_reward)

            iters += 1

            performance_list.append(running_reward)

            t.set_description(f'Episode {i}')
            t.set_postfix(
                episode_reward=episode_reward, running_reward=running_reward)

            # Show average episode reward every 10 episodes
            if i % 10 == 0:
                pass  # print(f'Episode {i}: average reward: {avg_reward}')

            if running_reward > reward_threshold and i >= min_episodes_criterion:
                break
    if performance_list[-1] < reward_threshold:
        return train_rl()
    return performance_list

Analyze network performance:

In [ ]:
# Number of each network to train and then average
num_networks_trained = 1

# Lists that will store the running reward data across episodes
repnet_performances = []
dist_repnet_performances = []
rl_performances = []

if __name__ == '__main__':
    # The loop to train each of the three types of networks num_networks_trained amount of times
    for i in range(num_networks_trained+1):
        if i % 10 == 0 and i > 0:
          print(f"{100*i/num_networks_trained}% done!")
          dist_repnet_data = [len(x) for x in dist_repnet_performances]


        # Different hyperparameters can be set using kwargs
        # repnet_perform = train_repnet()
        dist_repnet_perform = train_distributed_repnet(threshold_func=lambda x: 15, kill_time=75)
        rl_perform = train_rl()

        # Not recommended to plot a large number of networks at once
        # if i == 1:
        #   # Only use the labels the first time
        #   plt.plot(repnet_perform, color="magenta", label="REPNET")
        #   plt.plot(dist_repnet_perform, color="black", label="Distributed REPNET")
        #   plt.plot(rl_perform, color="teal", label="Normal RL")
        # else:
        #   plt.plot(repnet_perform, color="magenta")
        #   plt.plot(dist_repnet_perform, color="black")
        #   plt.plot(rl_perform, color="teal")

        # rl_performances.append(rl_perform)
        # repnet_performances.append(repnet_perform)
        dist_repnet_performances.append(dist_repnet_perform)

    # Get the number of episodes for each training instance each network required to achieve the threshold (195)
    # repnet_data = [len(x) for x in repnet_performances]
    dist_repnet_data = [len(x) for x in dist_repnet_performances]
    # rl_data = [len(x) for x in rl_performances]

    # Print the mean and standard deviation of each network across num_networks_trained number of each network
    # print(f"REPNET average # of weight updates: {np.mean(repnet_data)}, standard dev: {np.std(repnet_data)}")
    print(f"Distributed REPNET average # of weight updates: {np.mean(dist_repnet_data)}, standard dev: {np.std(dist_repnet_data)}")
    # print(f"RL average # of weight updates {np.mean(rl_data)}, standard dev: {np.std(rl_data)}")

    # plt.title("REPNET versus normal actor-critic RL on the CartPole env:")
    # # plt.title("REPNET and distributed variant versus normal actor-critic RL on the CartPole env:")
    # plt.xlabel("Running reward")
    # plt.ylabel("Cumulative training episodes")
    # plt.legend()
    # plt.show()

10.0% done!
Distributed REPNET average # of weight updates: 167.0, standard dev: 34.295772334210525
20.0% done!
Distributed REPNET average # of weight updates: 166.25, standard dev: 35.87321424126921
30.0% done!
Distributed REPNET average # of weight updates: 170.66666666666666, standard dev: 37.568899667440654
40.0% done!
Distributed REPNET average # of weight updates: 176.225, standard dev: 37.35738715435008
50.0% done!
Distributed REPNET average # of weight updates: 171.2, standard dev: 42.61689805699143
60.0% done!
Distributed REPNET average # of weight updates: 174.93333333333334, standard dev: 40.93200323571873
70.0% done!
Distributed REPNET average # of weight updates: 176.68571428571428, standard dev: 41.64562843020788
80.0% done!
Distributed REPNET average # of weight updates: 178.6375, standard dev: 42.65918533856454
90.0% done!
Distributed REPNET average # of weight updates: 176.95555555555555, standard dev: 41.255548074332744
Distributed REPNET average # of weight updates: 